In [ ]:
import matplotlib.pyplot as plt #for plotting
from matplotlib.ticker import EngFormatter #Formater of Engineering notation string
import matplotlib.ticker as mtick  #To draw vertical lines to plots
import numpy as np #numpy
from quantiphy import Quantity #to format numbers 
import mpld3 #interactive plots
#To add the toolbar to plots

## Introduction

* This notebook aims to exercise the block diagram of the OFDM technique for a given number of carriers to be transmitted over a channel;
* Initially, the modulation chosen will be BPSK for each carrier. In the future, QAM will also be applied;
* The application will map and input char word and will codify it into OFDM in terms of ASCII.

## System setup

The block diagram below describes the way one apply OFDM into a communication system:

<img src="OFDM_block_diagram.png" width="400" height="300" />

### Description

From the input data up to the D/A converter, everything is computed digitally in the transmitter, by using a digital signal processor or a microcontroller unit. Data is formatted into an OFDM symbol through an IFFT algorithm. The process is described as follows. 

The modulator selects the symbols out of a constellation to represent the input data. A Serial to Parallell converter delivers the symbols into an IFFT algorithm, that transforms the amplitudes and phases of the carriers into a time-domain signal. The signal is then formatted with a well-defined pulse (e.g., a raised-cosine pulse) that is bandlimited.


### A note on the D/A Converter

It is worth expanding what happens during the D/A conversion. A zero-order-hold (ZOH) D/A converter may be used to perform the conversion. If one is using a D/A that receives data through SPI communication, this would imply into a making the D/A having a higher sampling frequency that can be supported by the OFDM system.

**This if it is assumed that the D/A's settling time is lower than the SPI's delay time. But this is a topic for another notebook.**

For example, for a 10 bit ZOH D/A converter using SPI to communicate to the MCU responsibile for the OFDM, each symbol would require 10 SPI clock cycles to feed the entire bit word of one single sample of the OFDM signal. 

Thus,

$ T_{s} = 10T_{\mathrm{SPI}}$,

in which $T_{s}$ is the effective sampling frequency seen by the OFDM system. 

In other words, $T_s$ would be the sampling period of a weighted ZOH output signal, such as the one seen below:

<img src="ZOH.png" width="400" height="300" />

In conclusion, each change of the ZOH amplitude signal in red above represents a 10 bit word sent by the SPI bus. And $T_s$ represents the update time of the ZOH (its sampling frequency).


Moreover, for a SPI bus of $1\;\mathrm{\mu s}$ clock period, it would require **10 $\mathrm{\mu s}$ to feed the bitstream corresponding to one digital sample of the OFDM symbol**. For a 32 carriers OFDM, this would imply into a time-domain signal of 32 samples, and a 32 length FFT algorithm. **Meaning, 32 ZOH period to format one single OFDM pulse.**

Therefore, the OFDM symbol period would be of

$T_{symbol} = 32T_{s} = 32 \times 10\;\mathrm{\mu s} = 320\;\mathrm{\mu s}$

Furthermore, since the effective sampling period of the system is $10\;\mathrm{\mu s}$ (10 SPI periods), Nyquist tells us that the maximum allowable frequency we can reproduce would be $ 1/(2 \times T_s)$, or 500 kHz in our case, with a 10 bit resolution.

Hence, the last reliable carrier of a bandbase OFDM system would be 500 kHz, with the minimum recovery resolution.

### Communication parameters

Based on the arguments provided below, our OFDM scenario is aligned with the following parameters:

* SPI Bus clock of 1 MHz, or 1 $T_{\mathrm{SPI}} = 1\;\mathrm{\mu s}$;

* ZOH resolution of 10 bits; 

* ZOH Sampling period of $ T_{s} = 10T_{\mathrm{SPI}}$;

* Symbol period of 32 samples (32 carriers);

* Maximum allowable carrier to be recovered: 500 kHz;

* Desired modulation scheme of each carrier: Binary phase shift keying (BPSK);

## First OFDM Requirements

* One important point is that the number of carriers define the length of the OFDM symbol;

In [ ]:
ncarr = 8 #defines the number of carriers to be used
numSymbols = 2 #number of OFDM symbols
numPoints = ncarr*numSymbols #Total number of points 



In [ ]:
teste = np.array([1,-1,1,1,-1,-1,-1,-1])
x = np.fft.ifft(teste)
print(x)